In [1]:
from datetime import datetime as dt
from collections import defaultdict
import time
import numpy as np
import pandas as pd
import math
from scipy import stats
from scipy.signal import argrelextrema
from scipy.signal import savgol_filter
import statsmodels.api as sm
from sympy import *
import pytz
import itertools
import matplotlib.pyplot as plt
from itertools import combinations
from itertools import product
from itertools import groupby
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5

In [2]:
if not mt5.initialize(path="C:/JoeC/MetaTrader 5/terminal64.exe",login=1049732, server="TradersWay-Demo",password="r5AH8znU"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [2]:
def levelgrab(data, candles, savgol, cmdata, g):
    
    cndl = len(data) - candles
    S = data[cndl:]
    D = cmdata[cndl:]
    sav = int(savgol)
    SCM = np.average(abs(D['close'] - D['open']))
    y = S['close']
    try:
        y = savgol_filter(y, sav, 3)
        LVL = np.concatenate((y[argrelextrema(y, np.greater)[0]],y[argrelextrema(y, np.less)[0]]))
        B = list(combinations(LVL, 2))
        Ba = np.array([x for x, y in B])
        B = np.array([abs(x - y) for x, y in B])
        Bb = np.array([np.where(x <= SCM, 1, 0) for x in B])
        T = [(Ba[i], Bb[i]) for i in range(0, len(Ba))] 
        Toutput = defaultdict(int) 
        for k, v in T: 
            Toutput[k] += v 
        table = list(Toutput.items())
        tablevs = [x for x, y in table]
        tabpings = [y for x, y in table]
        tabmax = argrelextrema(np.array(tabpings), np.greater)
        tabmax2 = list(tabmax[0])
        levs = np.array(tablevs)[tabmax2]
        levs = list(levs)
        levs.sort(reverse=True)
        levs = list(levs)
        if g == 1:
            mindist = min([abs(j-i) for i, j in zip(levs[:-1], levs[1:])])
            while mindist < SCM:
                i = 0
                h = -1
                j = 1
                for c in range(0, len(levs)):
                    if j == len(levs):
                        j = -1
                    if abs(levs[i]-levs[h]) < SCM:
                        levs[i] = 0
                    elif abs(levs[i]-levs[j]) < SCM:
                        levs[i] = levs[i] - abs(levs[i]-levs[j])/2
                    i += 1
                    h += 1
                    j += 1
                levs = [v for v in levs if v != 0]
                mindist = min([abs(a-b) for a, b in zip(levs[:-1], levs[1:])])
        levs = tuple(levs)
    except ValueError:
        levs = 0

    return levs


def scopetrend(data, pair, candles, thresh):
    
    cndl = len(data) - candles
    F = data[cndl:]
    x1 = np.linspace(1, candles, num=candles)
    yclose = F['close']
    if "JPY" in pair:
        y1 = yclose*100
    elif "Crude" in pair:
        y1 = yclose*10
    elif "MXN" in pair:
        y1 = yclose*1000
    elif "DKK" in pair:
        y1 = yclose*1000
    elif "SEK" in pair:
        y1 = yclose*1000
    elif "NOK" in pair:
        y1 = yclose*1000
    elif "BTC" in pair:
        y1 = yclose*10
    elif "ETC" in pair:
        y1 = yclose*1000
    elif "ZEC" in pair:
        y1 = yclose*100
    elif "XMR" in pair:
        y1 = yclose*100
    elif "LTC" in pair:
        y1 = yclose*100
    elif "DSH" in pair:
        y1 = yclose*100
    elif "ETH" in pair:
        y1 = yclose*100
    elif "BCH" in pair:
        y1 = yclose*100
    elif "XAU" in pair:
        y1 = yclose*100
    else:
        y1 = yclose*10000
    slope, intercept, r_value, p_value, std_err = stats.linregress(x1,y1)
    trend = slope
    theta = abs(math.degrees(math.atan(trend)))
    if theta < thresh:
        scope_theta = 0
    else:
        scope_theta = 1
    if trend > 0:
        scope_trend = 1
    elif trend < 0:
        scope_trend = -1
    else:
        scope_trend = 0
    
    return scope_trend, scope_theta


def insttrend(data, pair, candles, thresh):
    
    cndl = len(data) - candles
    DT = data[cndl:]
    x21 = np.linspace(1, candles, num=candles)
    y2close = DT['close']
    if "JPY" in pair:
        y21 = y2close*100
    elif "Crude" in pair:
        y21 = y2close*10
    elif "MXN" in pair:
        y21 = y2close*1000
    elif "DKK" in pair:
        y21 = y2close*1000
    elif "SEK" in pair:
        y21 = y2close*1000
    elif "NOK" in pair:
        y21 = y2close*1000
    elif "BTC" in pair:
        y21 = y2close*10
    elif "ETC" in pair:
        y21 = y2close*1000
    elif "ZEC" in pair:
        y21 = y2close*100
    elif "XMR" in pair:
        y21 = y2close*100
    elif "LTC" in pair:
        y21 = y2close*100
    elif "DSH" in pair:
        y21 = y2close*100
    elif "ETH" in pair:
        y21 = y2close*100
    elif "BCH" in pair:
        y21 = y2close*100
    elif "XAU" in pair:
        y21 = y2close*100
    else:
        y21 = y2close*10000
    slope, intercept, r_value, p_value, std_err = stats.linregress(x21,y21)
    trend = slope
    theta = abs(math.degrees(math.atan(trend)))
    if theta < thresh:
        inst_theta = 0
    else:
        inst_theta = 1
    if trend > 0:
        inst_trend = 1
    elif trend < 0:
        inst_trend = -1
    else:
        inst_trend = 0

    return inst_trend, inst_theta


def microtrend(data, pair, candles, thresh):
    
    cndl = len(data) - candles
    DL = data[cndl:]
    x31 = np.linspace(1, candles, num=candles)
    y3close = DL['close']
    if "JPY" in pair:
        y31 = y3close*100
    elif "Crude" in pair:
        y31 = y3close*10
    elif "MXN" in pair:
        y31 = y3close*1000
    elif "DKK" in pair:
        y31 = y3close*1000
    elif "SEK" in pair:
        y31 = y3close*1000
    elif "NOK" in pair:
        y31 = y3close*1000
    elif "BTC" in pair:
        y31 = y3close*10
    elif "ETC" in pair:
        y31 = y3close*1000
    elif "ZEC" in pair:
        y31 = y3close*100
    elif "XMR" in pair:
        y31 = y3close*100
    elif "LTC" in pair:
        y31 = y3close*100
    elif "DSH" in pair:
        y31 = y3close*100
    elif "ETH" in pair:
        y31 = y3close*100
    elif "BCH" in pair:
        y31 = y3close*100
    elif "XAU" in pair:
        y31 = y3close*100
    else:
        y31 = y3close*10000
    slope, intercept, r_value, p_value, std_err = stats.linregress(x31,y31)
    trend = slope
    theta = abs(math.degrees(math.atan(trend)))
    if theta < thresh:
        micro_theta = 0
    else:
        micro_theta = 1
    if trend > 0:
        micro_trend = 1
    elif trend < 0:
        micro_trend = -1
    else:
        micro_trend = 0

    return micro_trend, micro_theta


def OpenLevTestA(levels, data, candles, Q, R, S):
    
    cndl = len(data) - candles
    D = data[cndl:]
    scndl = len(Q) - S
    QR = Q[scndl:]
    CM = np.average(abs(QR['close'] - QR['open'])) / R
    P = D[0]

    OTESTLVL = 0
    for u in levels:
        if abs(P[1]-u) <= CM:
            OTESTLVL = u
            
    return OTESTLVL


def CloseLevTestA(levels, data, candles, Q, R, S):
    
    cndl = len(data) - candles
    D = data[cndl:]
    scndl = len(Q) - S
    QR = Q[scndl:]
    CM = np.average(abs(QR['close']-QR['open'])) / R
    P = D[0]
    CTESTLVL = 0
    for u in levels:
        if abs(P[4]-u) <= CM:
            CTESTLVL = u
            
    return CTESTLVL


def BreakLevTestA(levels, data, candles, Q, R, S, OOTT):
    
    cndl = len(data) - candles
    D = data[cndl:]
    scndl = len(Q) - S
    QR = Q[scndl:]
    CM = np.average(abs(QR['close']-QR['open'])) / R
    P = D[0]
    BREAKLVL = 0
    for u in levels:
        if P['close'] - P['open'] > 0:
            if P[4] > (u+CM):
                if P[1] < u:
                    BREAKLVL = u
                elif OOTT == u:
                    BREAKLVL = u
        elif P['close'] - P['open'] > 0:
            if P[4] < (u-CM):
                if P[1] > u:
                    BREAKLVL = u
                elif OOTT == u:
                    BREAKLVL = u
    
    return BREAKLVL


def OpenLevTestB(levels, data, candles, Q, R, S):
    
    cndl = len(data) - candles
    D = data[cndl:]
    scndl = len(Q) - S
    QR = Q[scndl:]
    CM = np.average(abs(QR['close'] - QR['open'])) / R
    P = D[1]

    OTESTLVL = 0
    for u in levels:
        if abs(P[1]-u) <= CM:
            OTESTLVL = u
            
    return OTESTLVL


def CloseLevTestB(levels, data, candles, Q, R, S):
    
    cndl = len(data) - candles
    D = data[cndl:]
    scndl = len(Q) - S
    QR = Q[scndl:]
    CM = np.average(abs(QR['close']-QR['open'])) / R
    P = D[1]
    CTESTLVL = 0
    for u in levels:
        if abs(P[4]-u) <= CM:
            CTESTLVL = u
            
    return CTESTLVL


def BreakLevTestB(levels, data, candles, Q, R, S, OOTT):
    
    cndl = len(data) - candles
    D = data[cndl:]
    scndl = len(Q) - S
    QR = Q[scndl:]
    CM = np.average(abs(QR['close']-QR['open'])) / R
    P = D[1]
    BREAKLVL = 0
    for u in levels:
        if P['close'] - P['open'] > 0:
            if P[4] > (u+CM):
                if P[1] < u:
                    BREAKLVL = u
                elif OOTT == u:
                    BREAKLVL = u
        elif P['close'] - P['open'] > 0:
            if P[4] < (u-CM):
                if P[1] > u:
                    BREAKLVL = u
                elif OOTT == u:
                    BREAKLVL = u
    
    return BREAKLVL


def OpenLevTestC(levels, data, candles, Q, R, S):
    
    cndl = len(data) - candles
    D = data[cndl:]
    scndl = len(Q) - S
    QR = Q[scndl:]
    CM = np.average(abs(QR['close'] - QR['open'])) / R
    P = D[2]

    OTESTLVL = 0
    for u in levels:
        if abs(P[1]-u) <= CM:
            OTESTLVL = u
            
    return OTESTLVL


def CloseLevTestC(levels, data, candles, Q, R, S):
    
    cndl = len(data) - candles
    D = data[cndl:]
    scndl = len(Q) - S
    QR = Q[scndl:]
    CM = np.average(abs(QR['close']-QR['open'])) / R
    P = D[2]
    CTESTLVL = 0
    for u in levels:
        if abs(P[4]-u) <= CM:
            CTESTLVL = u
            
    return CTESTLVL


def BreakLevTestC(levels, data, candles, Q, R, S, OOTT):
    
    cndl = len(data) - candles
    D = data[cndl:]
    scndl = len(Q) - S
    QR = Q[scndl:]
    CM = np.average(abs(QR['close']-QR['open'])) / R
    P = D[2]
    BREAKLVL = 0
    for u in levels:
        if P['close'] - P['open'] > 0:
            if P[4] > (u+CM):
                if P[1] < u:
                    BREAKLVL = u
                elif OOTT == u:
                    BREAKLVL = u
        elif P['close'] - P['open'] > 0:
            if P[4] < (u-CM):
                if P[1] > u:
                    BREAKLVL = u
                elif OOTT == u:
                    BREAKLVL = u
    
    return BREAKLVL


def Concavity(data, candles, savgol, sel):
    
    cndl = len(data) - candles
    Ho = data[cndl:]
    H = Ho['close']
    hlin = np.linspace(1,candles,num=candles)
    sav = int(savgol)
    H = savgol_filter(H, sav, 3)
    SLOPE = []
    i = 0
    for t in hlin:
        ii = i - 1
        if i == 0:
            SLOPE.append(0)
        else:
            slope = H[i] - H[ii]
            SLOPE.append(slope)
        i += 1
    SLOPE2 = []
    i = 0
    for t in hlin:
        ii = i - 1
        if i == 0:
            SLOPE2.append(0)
        else:
            slope2 = SLOPE[i] - SLOPE[ii]
            SLOPE2.append(slope2)
        i += 1
    CONCAVE = []
    i = 0
    for t in hlin:
        if SLOPE2[i] == 0:
            CONCAVE.append('INFL')
        elif SLOPE2[i] > 0:
            CONCAVE.append('UP')
        else:
            CONCAVE.append('DOWN')
        i += 1
        
    select = int(sel)
    AXCONCAVE = CONCAVE[select]
    
    return AXCONCAVE


def LongFramer(levels, ggg, CCTT, shrink):
    
    if levels is not None:
        if levels != 0:
            if CCTT != 0:
                entry = ggg['close'][-1]
                ltime = (pd.to_datetime(ggg['time'][-1], unit='s'))
                cend = int(len(levels)-1)
                clindex = levels.index(CCTT)
                tpindex = int(clindex - 1)
                slindex = int(clindex + 1)
                if clindex == 0:
                    stoploss = levels[slindex]
                    sldist = entry - stoploss
                    takeprofit = entry + sldist
                elif clindex == cend:
                    takeprofit = levels[tpindex]
                    tpdist = takeprofit - entry
                    stoploss = entry - tpdist
                else:
                    takeprofit = levels[tpindex]
                    stoploss = levels[slindex]
                tpdist = takeprofit - entry
                takeprofit = entry + tpdist*shrink
                tpdist = takeprofit - entry
                sldist = entry - stoploss
                if sldist > tpdist:
                    stoploss = entry - tpdist
                lentry = entry
                ltakeprofit = takeprofit
                lstoploss = stoploss
            else:
                ltime = int(0)
                lentry = 0
                ltakeprofit = 0
                lstoploss = 0
    
    return ltime, lentry, ltakeprofit, lstoploss


def ShortFramer(levels, ggg, CCTT, shrink):
    
    if levels is not None:
        if levels != 0:
            if CCTT != 0:
                entry = ggg['close'][-1]
                stime = (pd.to_datetime(ggg['time'][-1], unit='s'))
                cend = int(len(levels)-1)
                clindex = levels.index(CCTT)
                tpindex = int(clindex + 1)
                slindex = int(clindex - 1)
                if clindex == 0:
                    takeprofit = levels[tpindex]
                    tpdist = entry - takeprofit
                    stoploss = entry + tpdist
                elif clindex == cend:
                    stoploss = levels[slindex]
                    sldist = stoploss - entry
                    takeprofit = entry - sldist
                else:
                    takeprofit = levels[tpindex]
                    stoploss = levels[slindex]
                tpdist = entry - takeprofit
                takeprofit = entry - tpdist*shrink
                tpdist = entry - takeprofit
                sldist = stoploss - entry
                if sldist > tpdist:
                    stoploss = entry + tpdist
                sentry = entry
                stakeprofit = takeprofit
                sstoploss = stoploss
            else:
                stime = int(0)
                sentry = 0
                stakeprofit = 0
                sstoploss = 0
        
    return stime, sentry, stakeprofit, sstoploss




## BEGIN ANALYSIS


# DATA PRIMER


print("Initializing...")

start = dt.now()

candles = 1000

pairs = []
M1_DATA = {}
M5_DATA = {}
M15_DATA = {}
M30_DATA = {}
H1_DATA = {}
H2_DATA = {}
H3_DATA = {}
H4_DATA = {}
H8_DATA = {}
D1_DATA = {}
symbols=mt5.symbols_get()
for s in symbols:
    pairs.append(s.name)
    M1_DATA[s.name] = mt5.copy_rates_from_pos(s.name, mt5.TIMEFRAME_M1, 0, candles)
    M5_DATA[s.name] = mt5.copy_rates_from_pos(s.name, mt5.TIMEFRAME_M5, 0, candles)
    M15_DATA[s.name] = mt5.copy_rates_from_pos(s.name, mt5.TIMEFRAME_M15, 0, candles)
    M30_DATA[s.name] = mt5.copy_rates_from_pos(s.name, mt5.TIMEFRAME_M30, 0, candles)
    H1_DATA[s.name] = mt5.copy_rates_from_pos(s.name, mt5.TIMEFRAME_H1, 0, candles)
    H2_DATA[s.name] = mt5.copy_rates_from_pos(s.name, mt5.TIMEFRAME_H2, 0, candles)
    H3_DATA[s.name] = mt5.copy_rates_from_pos(s.name, mt5.TIMEFRAME_H3, 0, candles)
    H4_DATA[s.name] = mt5.copy_rates_from_pos(s.name, mt5.TIMEFRAME_H4, 0, candles)
    H8_DATA[s.name] = mt5.copy_rates_from_pos(s.name, mt5.TIMEFRAME_H8, 0, candles)
    D1_DATA[s.name] = mt5.copy_rates_from_pos(s.name, mt5.TIMEFRAME_D1, 0, candles)
    


# LEVELS PARAMETERS

LevelGrab_chart = [D1_DATA, H8_DATA, H4_DATA, H3_DATA, H2_DATA, H1_DATA, M30_DATA]
LevelGrab_candles = [1000, 1500, 2000]
# CONTROL (PEPPER) : 1000 CANDLES ON THE 4-HOUR
LevelGrab_savgol = [7, 13, 21, 31, 51, 71]
# CONTROL : none
LevelGrab_SCM = [D1_DATA, H8_DATA, H4_DATA, H3_DATA, H2_DATA, H1_DATA, M30_DATA, M15_DATA, M5_DATA]
# CONTROL : none
LevelGrab_wheel = [0, 1]
# CONTROL : 1


# SCOPETREND PARAMETERS

ScopeTrend_chart = [H1_DATA, M30_DATA, M15_DATA, M5_DATA]
ScopeTrend_candles = [150, 200, 400, 600, 800]
# CONTROL : 200 CANDLES ON THE 15-MIN
ScopeTheta_threshold = [30, 50, 60, 70, 80]
# CONTROL : 5


# INSTTREND PARAMETERS

InstTrend_chart = [M15_DATA, M5_DATA, M1_DATA]
InstTrend_candles = [75, 100, 150, 250, 350]
# CONTROL : 168 CANDLES ON THE 5-MIN
InstTheta_threshold = [30, 50, 60, 70, 80]


# MICROTREND PARAMETERS

MicroTrend_chart = [M5_DATA, M1_DATA]
MicroTrend_candles = [3, 7, 15, 25]
# CONTROL (PEPPER) : 10 CANDLES ON THE 1-MIN
MicroTheta_threshold = [30, 50, 60, 70, 80]
# CONTROL : none


# LEV TEST PARAMETERS

LevTest_chart = [H4_DATA, H2_DATA, H1_DATA, M30_DATA, M15_DATA, M5_DATA]
LevTest_Q = [H4_DATA, H2_DATA, H1_DATA, M30_DATA, M15_DATA, M5_DATA]
LevTest_R = [1,2,3]
LevTest_S = [100, 150, 250, 350]
# CONTROL : AVERAGE CANDLE ON 5 MIN


# CONCAVITY PARAMETERS

Concavity_charts = [H8_DATA, H4_DATA, H3_DATA, H2_DATA, H1_DATA, M30_DATA]
Concavity_candles = [25, 32, 48]
# CONTROL : 25 CANDLES ON THE 2-HOUR
Concavity_savgol = [7, 13, 21]
Concavity_selection = [-1, -2, -3, -5]
# CONTROL : SAVGOL 13 SELECTION [-1]


# FRAMING PARAMETERS

Framer_shrink = [0.75, 0.85, 0.9, 0.95, 1]


# COLUMN CREATOR




K = []
levs_chart = []
levs_candles = []
levs_savgol = []
levs_SCM = []
levs_wheel = []
scope_chart = []
scope_candles = []
scope_thresh = []
inst_chart = []
inst_candles = []
inst_thresh = []
micro_chart = []
micro_candles = []
micro_thresh = []
levstest_chart = []
levstest_Q = []
levstest_R = []
levstest_S = []
concavity_charts = []
concavity_candles = []
concavity_savgol = []
concavity_select = []
framer_shrink = []
for la in LevelGrab_chart:
    levs_chart.append("LevsChart_"+str(la))
    for lc in LevelGrab_candles:
        levs_candles.append("LevsCandles_"+str(lc))
        for ld in LevelGrab_savgol:
            levs_savgol.append("LevsSavgol_"+str(ld))
            for le in LevelGrab_SCM:
                levs_SCM.append("LevsSCM_"+str(le))
                for lg in LevelGrab_wheel:
                    levs_wheel.append("LevsWheel_"+str(lg))
                    for sa in ScopeTrend_chart:
                        scope_chart.append("ScopeChart_"+str(sa))
                        for sc in ScopeTrend_candles:
                            scope_candles.append("ScopeCandles_"+str(sc))
                            for sd in ScopeTheta_threshold:
                                scope_thresh.append("ScopeThresh_"+str(sd))
                                for ia in InstTrend_chart:
                                    inst_chart.append("InstChart_"+str(ia))
                                    for ic in InstTrend_candles:
                                        inst_candles.append("InstCandles_"+str(ic))
                                        for icd in InstTheta_threshold:
                                            inst_thresh.append("InstThresh_"+str(icd))
                                            for ma in MicroTrend_chart:
                                                micro_chart.append("MicroChart_"+str(ma))
                                                for mc in MicroTrend_candles:
                                                    micro_candles.append("MicroCandles_"+str(mc))
                                                    for md in MicroTheta_threshold:
                                                        micro_thresh.append("MicroThresh_"+str(md))
                                                        for olaa in LevTest_chart:
                                                            levstest_chart.append("LevsTestChart_"+str(olaa))
                                                            for olac in LevTest_Q:
                                                                levstest_Q.append("LevsTestQ_"+str(olac))
                                                                for olae in LevTest_R:
                                                                    levstest_R.append("LevsTestR_"+str(olae))
                                                                    for olaf in LevTest_S:
                                                                        levstest_S.append("LevsTestS_"+str(olaf))
                                                                        for ca in Concavity_chart:
                                                                            concavity_charts.append("ConcavCharts_"+str(ca))
                                                                            for cc in Concavity_candles:
                                                                                concavity_candles.append("ConcavCandles_"+str(cc))
                                                                                for cd in Concavity_savgol:
                                                                                    concavity_savgol.append("ConcavSavgol_"+str(cd))
                                                                                    for ce in Concavity_selection:
                                                                                        concavity_select.append("ConcavSelect_"+str(ce))
                                                                                        for fa in Framer_shrink:
                                                                                            framer_shrink.append("FramerShrink_"+str(fa))

K.append(levs_chart)
K.append(levs_candles)
K.append(levs_savgol)
K.append(levs_SCM)
K.append(levs_wheel)
K.append(scope_chart)
K.append(scope_candles)
K.append(scope_thresh)
K.append(inst_chart)
K.append(inst_candles)
K.append(inst_thresh)
K.append(micro_chart)
K.append(micro_candles)
K.append(micro_thresh)
K.append(levstest_chart)
K.append(levstest_Q)
K.append(levstest_R)
K.append(levstest_S)
K.append(concavity_charts)
K.append(concavity_candles)
K.append(concavity_savgol)
K.append(concavity_select)
K.append(framer_shrink)


                
G = []
for p in pairs:
    for la in LevelGrab_chart:
        for lc in LevelGrab_candles:
            for ld in LevelGrab_savgol:
                for le in LevelGrab_SCM:
                    for lf in pairs:
                        for lg in LevelGrab_wheel:
                            levs = levelgrab(la[p], lc, ld, le[lf], lg)
                            for sa in ScopeTrend_chart:
                                for sc in ScopeTrend_candles:
                                    for sd in ScopeTheta_threshold:
                                        scope_trend, scope_theta = scopetrend(sa[p], p, sc, sd)
                                        for ia in InstTrend_chart:
                                            for ic in InstTrend_candles:
                                                for icd in InstTheta_threshold:
                                                    inst_trend, inst_theta = insttrend(ia[p], p, ic, icd)
                                                    for ma in MicroTrend_chart:
                                                        for mc in MicroTrend_candles:
                                                            for md in MicroTheta_threshold:
                                                                micro_trend, micro_theta = microtrend(ma[p], p, mc, md)
                                                                for olaa in LevTest_chart:
                                                                    for olac in LevTest_Q:
                                                                        for olad in pairs:
                                                                            for olae in LevTest_R:
                                                                                for olaf in LevTest_S:
                                                                                    OTESTLVLA = OpenLevTestA(levs, olaa[p], 4, olac[olad], olae, olaf)
                                                                                    CTESTLVLA = CloseLevTestA(levs, olaa[p], 4, olac[olad], olae, olaf)
                                                                                    BREAKLVLA = BreakLevTestA(levs, olaa[p], 4, olac[olad], olae, olaf, OTESTLVLA)
                                                                                    OTESTLVLB = OpenLevTestB(levs, olaa[p], 4, olac[olad], olae, olaf)
                                                                                    CTESTLVLB = CloseLevTestB(levs, olaa[p], 4, olac[olad], olae, olaf)
                                                                                    BREAKLVLB = BreakLevTestB(levs, olaa[p], 4, olac[olad], olae, olaf, OTESTLVLB)
                                                                                    OTESTLVLC = OpenLevTestC(levs, olaa[p], 4, olac[olad], olae, olaf)
                                                                                    CTESTLVLC = CloseLevTestC(levs, olaa[p], 4, olac[olad], olae, olaf)
                                                                                    BREAKLVLC = BreakLevTestC(levs, olaa[p], 4, olac[olad], olae, olaf, OTESTLVLC)
                                                                                    for ca in Concavity_chart:
                                                                                        for cc in Concavity_candles:
                                                                                            for cd in Concavity_savgol:
                                                                                                for ce in Concavity_selection:
                                                                                                    AXCONCAVE = Concavity(ca[p], cc, cd, ce)
                                                                                                    for fa in Framer_shrink:
                                                                                                        lentry, ltakeprofit, lstoploss = LongFramer(levs, M1_DATA[p], CTESTLVLC, fa)
                                                                                                        sentry, stakeprofit, sstoploss = ShortFramer(levs, M1_DATA[p], CTESTLVLC, fa)
                                                                                                        G.append(la,lb,lc,ld,le,lg,sa,sc,sd,ia,ic,icd,ma,mc,md,olaa,olac,olae,olaf,ca,cc,cd,ce,fa,ltime,lentry,ltakeprofit,lstoploss,stime,sentry,stakeprofit,sstoploss)
                
Output_Table = pd.DataFrame(G)
end = dt.now()
elapsed = end - start
elapsed = str(elapsed.total_seconds()) + '   seconds'
print(candles," candles, elapsed time: ",elapsed)

Initializing...


NameError: name 'dt' is not defined

In [ ]:
## ADDITIONAL PARAMETERS TO ADD TO THE MIX:

## ICHIMOKU BLUE LINE

## ICHIMOKU CLOUD*

## RELATIVE STRENGTH INDEX (RSI)

In [ ]:
## OUR OUTPUT G IS A LIST OF TUPLES REPRESENTING ALL OF THE RELEVENT INPUTS AND THE TRADE OUTPUTS
## THIS MEANS G IS A LIST OF TUPLES WITH INPUTS AND THE TRADES THAT THOSE INPUTS DESIRE TO PLACE
## WE NEED TO NOW BUILD A SEPARATE FUNCTION - AND MAYBE CONCATENATE THE TWO IN THE FUTUTE - 
##          THAT CAN INTERPRET THE TRADE DATA FROM G AND OBSERVE NEW PRICE DATA TO CALCULATE PROFIT/LOSS
##                             WE WILL MEASURE THIS IN PIPS!! SKIP THE LOT CALCULATIONS AND SHIT

## SO, GOING FORWARD WE NEED TO BUILD THE TRADEWATCH FUNCTION, AND SEE ABOUT MAKING IT ALL ONE CALCULATION